<a href="https://colab.research.google.com/github/yukinaga/min_gen_agent_app/blob/main/section_2/02_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像生成アプリ

In [ ]:
#================================================================
# 1. 必要なライブラリのインストール
#================================================================
# OpenAIのAPIを利用するためのライブラリと、
# Web UIを簡単に作成するためのGradioをインストールします。
# ! は、Colaboratoryのセルでターミナルコマンドを実行するためのおまじないです。
!pip install openai gradio

#================================================================
# 2. ライブラリのインポートとAPIキーの設定
#================================================================
import openai  # OpenAIライブラリ
import gradio as gr  # Gradioライブラリ
from google.colab import userdata  # Colaboratoryのシークレットを読み込むためのライブラリ

# シークレットに保存したAPIキーを安全に読み込みます。
# 'OPENAI_API_KEY' は、先ほどシークレットに設定した「名前」と一致している必要があります。
try:
    openai.api_key = userdata.get('OPENAI_API_KEY')
    # OpenAIライブラリの新しいバージョン(v1.0.0以降)では、このようにクライアントを作成します。
    client = openai.OpenAI(api_key=openai.api_key)
except Exception as e:
    # APIキーの読み込みに失敗した場合にエラーメッセージを表示します。
    print("APIキーの読み込みに失敗しました。Colaboratoryのシークレット設定を確認してください。")
    print(f"エラー詳細: {e}")
    client = None # エラーが発生した場合はクライアントをNoneに設定

#================================================================
# 3. 画像生成を行う関数を定義
#================================================================
# この関数がDALL-Eを使って画像を生成する部分です。
# ユーザーが入力したテキスト（プロンプト）を受け取ります。
def generate_image(prompt):
    # APIキーが正しく設定されていない場合は、エラーメッセージを返します。
    if client is None:
        return None, "エラー: OpenAIクライアントが初期化されていません。APIキーの設定を確認してください。"

    # プロンプトが空の場合もエラーとします。
    if not prompt:
        return None, "プロンプトを入力してください。"

    try:
        # OpenAIのDALL-Eモデルを呼び出して画像を生成します。
        response = client.images.generate(
            model="dall-e-3",  # 使用するDALL-Eモデルを指定します。今回は `dall-e-3` を使用します。
            prompt=prompt,      # ユーザーが入力した画像の説明文（プロンプト）
            size="1024x1024",   # 生成する画像のサイズ。`dall-e-3` では 1024x1024, 1792x1024, 1024x1792 が選択可能です。
            quality="standard", # 画像の品質。"standard" または "hd" (高画質) が選択可能ですが、hdはコストが高くなります。
            n=1                 # 生成する画像の枚数。現在は1枚のみ対応しています。
        )

        # 生成された画像のURLを取得します。
        image_url = response.data[0].url
        # 成功メッセージと共に画像のURLを返します。
        return image_url, "画像を生成しました！"

    except openai.APIError as e:
        # OpenAI API固有のエラーが発生した場合
        print(f"OpenAI APIエラーが発生しました: {e}")
        return None, f"画像を生成できませんでした。APIエラー: {e}"
    except Exception as e:
        # その他のエラーが発生した場合
        print(f"画像生成中に予期せぬエラーが発生しました: {e}")
        return None, f"画像を生成できませんでした。エラー: {e}"

#================================================================
# 4. Gradioを使ってWeb UIを作成・起動
#================================================================
# gr.Blocks() を使うと、コンポーネントを自由に配置してリッチなUIを構築できます。
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # gr.Markdown() を使って、画面に説明文を表示します。
    gr.Markdown("# AI画像生成アプリ\nOpenAI DALL-EとGradioを使って、あなたのアイデアを画像にします！")

    # ユーザーがプロンプトを入力するテキストボックス
    # placeholderは、入力欄に薄く表示されるヒントテキストです。
    prompt_input = gr.Textbox(
        label="生成したい画像の説明（プロンプト）",
        placeholder="例: 青い空の下、白い雲と鮮やかな緑の丘がある牧歌的な風景",
        lines=3 # 複数行入力できるようにします
    )

    # 画像生成を開始するボタン
    generate_button = gr.Button("画像を生成")

    # 生成された画像を表示するコンポーネント
    # value=None は初期状態では何も表示しないことを意味します。
    generated_image = gr.Image(label="生成された画像", value=None)

    # ステータスやエラーメッセージを表示するテキストボックス
    status_output = gr.Textbox(label="ステータス", interactive=False) # interactive=Falseでユーザーが編集できないようにします

    # ボタンがクリックされたときの動作を定義します。
    # generate_buttonがクリックされると `generate_image` 関数が呼び出されます。
    # inputs: generate_image関数に渡す引数（prompt_inputの値）
    # outputs: generate_image関数の戻り値（画像URLとステータスメッセージ）を受け取るコンポーネント
    generate_button.click(
        fn=generate_image,
        inputs=[prompt_input],
        outputs=[generated_image, status_output]
    )

# demo.launch() で、作成したUIを起動します。
# debug=True にすると、エラー発生時に詳細な情報を確認できるため開発に便利です。
demo.launch(debug=True)